In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [ ]:
train_features=torch.load('train_features.pt')
train_labels=torch.load('train_labels.pt')

In [ ]:
train = torch.cat([train_features, train_labels.reshape(-1,1)],1)
print(train.shape)
train = train[torch.randperm(train.size(0))] #shuffling dataset, parameter down need to be updated
train_set = train[0:train.shape[0]*0.9,                            0:train.shape[1]-1]
train_label = train[0:train.shape[0]*0.9,                          train.shape[1]-1]
validation_set = train[train.shape[0]*0.9:train.shape[0]*0.95,     0:train.shape[1]-1]
validation_label = train[train.shape[0]*0.9:train.shape[0]*0.95,   train.shape[1]-1]
test_set = train[train.shape[0]*0.95:train.shape[0],               0:train.shape[1]-1]
test_label = train[train.shape[0]*0.95:train.shape[0],             train.shape[1]-1]

In [ ]:
#You may save them if you want
torch.save(train_set,'train_set.pt')
torch.save(train_label,'train_label.pt')
torch.save(validation_set,'validation_set.pt')
torch.save(validation_label,'validation_label.pt')
torch.save(test_set,'test_set.pt')
torch.save(test_label,'test_label.pt')
del train_features
del train_labels

In [ ]:
#I implement a MLP first, it is actually a Linear Regression
import torch.nn as nn
import torch.nn.functional as F


class MLP(nn.Module):
    def __init__(self, input_features):
        super(MLP, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(in_features=input_features,out_features=1)
        )
        

    def forward(self, x):
        x=self.layer1(x)
        return x

model = MLP(train_set.shape[1])
model = model.to(device)

In [ ]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

In [ ]:
import torch.optim as optim
learning_rate = 0.001
criterion = RMSELoss()
num_epochs = 10
BATCH_SIZE = 64

In [ ]:
from torch.utils.data import DataLoader, Dataset, TensorDataset
train_dataset = TensorDataset(train_set, train_label)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
print(len(train_loader))
all_losses = []

In [ ]:
def train(model, train_loader,criterion,num_epochs):
    optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate,weight_decay=1e-4)
    for epoch in range(num_epochs):
        epoch_loss = 0
        for i,(x, y) in enumerate(train_loader,0):
            x = x.to(device)
            y = y.to(device)
            y = torch.reshape(y,(-1,1))
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)#RMSE
            epoch_loss += loss
            loss.backward()
            optimizer.step()
        print(f"Epoch: {epoch+1} Loss:{epoch_loss/len(train_loader)}")
        all_losses.append(epoch_loss / len(train_loader))

In [ ]:
train(model, train_loader,criterion,num_epochs)

In [ ]:
plt.figure()
plt.plot(torch.Tensor.cpu(torch.tensor(all_losses)))

In [ ]:
PATH = 'linear.pth'
torch.save(model.state_dict(),PATH)

In [ ]:
#Use sklearn for something like gradient boosting or random forest

#Model Selection: Gradient Boosting,Random Forest,Extra Randomized Trees, SVM, Linear Regression, Logistic Regression, Neural Networks

In [ ]:
#TODO: build other models
#TODO: report loss on validation sets and tune parameters for each model
#TODO: run on test sets and report test loss
#TODO: predict travel time(test_features) and submit to kaggle